In [2]:
from pyspark.sql import SQLContext
import pyspark.sql.functions
from pyspark.sql.functions import avg,col
import pyspark.sql.types

In [3]:
sqlContext=SQLContext(sc)
form=raw_input("Select the input format\n1.csv\n2.json\n3.jbdc\n4.orc\n5.paraquet\n6.libsvm\n7.text ")
form=str(form)
path=raw_input("Enter the path ")
path=str(path)
df=sqlContext.read.format(form).option("header","True").option("inferSchema", "True").load(path)

Select the input format
1.csv
2.json
3.jbdc
4.orc
5.paraquet
6.libsvm
7.text csv
Enter the path /home/nymeria/Downloads/winemag-data_first150k.csv


In [4]:
df.dtypes

[('_c0', 'string'),
 ('country', 'string'),
 ('description', 'string'),
 ('designation', 'string'),
 ('points', 'string'),
 ('price', 'string'),
 ('province', 'string'),
 ('region_1', 'string'),
 ('region_2', 'string'),
 ('variety', 'string'),
 ('winery', 'string')]

In [6]:
cdtype=raw_input("Change any datatypes? (y/n) ")
if cdtype=="y":
    for x in df.columns:
        ch=raw_input("Change for "+x)
        if ch=="y":
            dt=raw_input("to data type? ")
            df = df.withColumn(x, df[x].cast(dt))
            
df.dtypes

Change any datatypes? (y/n) y
Change for _c0n
Change for countrym
Change for descriptionn
Change for designationn
Change for pointsn
Change for pricey
to data type? double
Change for provincen
Change for region_1n
Change for region_2n
Change for varietyn
Change for wineryn


[('_c0', 'int'),
 ('country', 'string'),
 ('description', 'string'),
 ('designation', 'string'),
 ('points', 'string'),
 ('price', 'double'),
 ('province', 'string'),
 ('region_1', 'string'),
 ('region_2', 'string'),
 ('variety', 'string'),
 ('winery', 'string')]

In [7]:
print("No. of rows in the dataframe="+str(df.count()))
print("No. of columns in the dataframe="+str(len(df.columns)))

No. of rows in the dataframe=150935
No. of columns in the dataframe=11


In [8]:
Nulls={}
for x in range(len(df.columns)):
            dfNoNulls=df.na.drop(how="all",subset=df.columns[x])
            Nulls[df.columns[x]]=((df.count()-dfNoNulls.count()))

In [9]:
print Nulls

{'province': 11, 'description': 1, 'designation': 45733, 'variety': 6, 'region_1': 25059, '_c0': 5, 'price': 13729, 'region_2': 89963, 'points': 6, 'winery': 17, 'country': 5}


In [10]:
threshd=input("Threshold value of missing rows to drop column? ")
for x in Nulls:
    if Nulls[x]>=threshd:
        print("dropping "+x)
        df=df.drop(x)

Threshold value of missing rows to drop column? 15000
dropping designation
dropping region_1
dropping region_2


In [11]:
print("Mean, Mode value for the missing value numeric columns")
nonnull=df.na.drop(how="all")

Mean, Mode value for the missing value numeric columns


In [12]:
for x in Nulls:
        if Nulls[x]>0 and Nulls[x]<=threshd:
            mm=raw_input("Mode or mean for column "+str(x))
            if mm == "mean":
                meanValue=nonnull.agg(avg(col(x))).first()[0]
                print(meanValue)
                df=df.na.fill(meanValue, [x])
            elif mm == "mode":
                import pandas as pd
                pdf=df.toPandas()
                moder=pdf.loc[:,x].mode()
                print(moder)
                val=input("Select value number to impute from the list above")
                df=df.na.fill(moder[val], [x])

Mode or mean for column provincemode
0    California
dtype: object
Select value number to impute from the list above0
Mode or mean for column descriptionmode
0    86-88 This could work as a rich wine, because ...
1    92-94 Barrel sample. A rounded wine, its tanni...
2    A little bit funky and unsettled when you pop ...
3    Powerful in Zinny character, this blend of Dry...
dtype: object
Select value number to impute from the list above0
Mode or mean for column varietymode
0    Chardonnay
dtype: object
Select value number to impute from the list above0
Mode or mean for column _c0mean
75464.5
Mode or mean for column pricemean
33.1310292553
Mode or mean for column pointsmode
0    87
dtype: object
Select value number to impute from the list above0
Mode or mean for column winerymode
0    Williams Selyem
dtype: object
Select value number to impute from the list above0
Mode or mean for column countrymode
0    US
dtype: object
Select value number to impute from the list above0


In [20]:
intp=[]
for x,y in df.dtypes:
    if y=='int':
        intp.append(x)
    elif y=='float':
         intp.append(x)
    elif y=='double':
        intp.append(x)
print intp

['_c0', 'price']


In [14]:
from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import Normalizer

Normopt=raw_input("Performing nomalization\nSelect type:\n1.$L^1$ norm\n2.$L^\infty$")

if Normopt is "$L^1$":
    for x in intp:
        normalizer = Normalizer(inputCol=x, outputCol=x, p=1.0)
        l1NormData = normalizer.transform(df)
elif Normopt is "$L^\infty$":
    for x in intp:
        lInfNormData = normalizer.transform(df, {normalizer.p: float(x)})

Performing nomalization
Select type:
1.$L^1$ norm
2.$L^\infty$$L^1$


In [21]:
a=intp
b=intp

li=[]

for x in a:
    for y in b:
        if x is not y:
            li.append(x)
            li.append(y)
            li.append(df.corr(x,y))
    
pair=[]

for x in range(0,len(li)/3):
    temp=[]
    temp.append(li[x*3])
    temp.append(li[(x*3)+1])
    temp.append(li[(x*3)+2])
    
    
    pair.append(temp)
    

In [22]:
threscorr=input("Enter the correlation threshold")
print("The feature pairs with corr coeff more than threshold are:")

for x in range(0,len(pair)):
    corr=pair[x][2]
    if corr>=threscorr:
        print[pair[x]]
        df=df.drop(pair[x][0])
        df=df.drop(pair[x][1])

Enter the correlation threshold0.9
The feature pairs with corr coeff more than threshold are:


In [ ]:
print("Exporting dataframe to",form)
pathsave=raw_input("The path to save to")
df.write.format(form).save(pathsave)

('Exporting dataframe to', 'csv')
